In [1]:
import os 
import math
from collections import defaultdict
import io
import time
import cv2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clt

import folium
import xyzservices.providers as xyz

from shapely.geometry import Polygon, box, MultiPolygon, LineString
from shapely.ops import unary_union
from scipy.spatial import distance_matrix

from selenium import webdriver
from PIL import Image

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
viirs = pd.read_csv("./data/viirs_s_npp_italy_2012_clustered.csv")

# Convert latitude and longitude to numeric after cleaning
viirs['latitude'] = pd.to_numeric(viirs['latitude'])
viirs['longitude'] = pd.to_numeric(viirs['longitude'])
viirs['datetime'] = pd.to_datetime(viirs['datetime'], errors='coerce')

In [3]:
# Define a colormap using matplotlib's color palette to generate unique colors
num_colors = len(viirs['fire_id'].unique())
colors = plt.cm.tab20(np.linspace(0, 1, num_colors))

# Map each ID to a specific color
id_to_color = {id_val: clt.rgb2hex(color[:3]) for id_val, color in zip(viirs['fire_id'].unique(), colors)}

In [4]:
# limit dataset to just calabria
# Set up the map boundaries
bounds = [[37.988279, 15.604679], [38.021622, 15.90]]
lat_min, lon_min = bounds[0]
lat_max, lon_max = bounds[1]
viirs = viirs[(viirs['latitude'] >= lat_min) & (viirs['latitude'] <= lat_max) & 
              (viirs['longitude'] >= lon_min) & (viirs['longitude'] <= lon_max)]



In [5]:
def create_square(lat, lon, side_length=375):
    """Create a square around a lat/lon point with a given side length in meters"""
    lat_diff = (side_length / 2) / 111320
    lon_diff = (side_length / 2) / (40075000 * math.cos(math.radians(lat)) / 360)

    return box(
        lon - lon_diff,  # minx
        lat - lat_diff,  # miny
        lon + lon_diff,  # maxx
        lat + lat_diff   # maxy
    )

def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate the great circle distance between two points in meters"""
    R = 6371000  # Earth radius in meters

    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c

def connect_nearby_squares(points_df, max_distance=750):
    """Create connections between nearby points"""
    coords = points_df[['latitude', 'longitude']].values

    # Calculate pairwise distances between all points
    distances = np.zeros((len(coords), len(coords)))
    for i in range(len(coords)):
        for j in range(i+1, len(coords)):
            dist = haversine_distance(coords[i,0], coords[i,1], coords[j,0], coords[j,1])
            distances[i,j] = dist
            distances[j,i] = dist

    # Find pairs of points that are within max_distance
    connections = []
    for i in range(len(coords)):
        for j in range(i+1, len(coords)):
            if distances[i,j] <= max_distance:
                connections.append((i, j))

    return connections

def merge_cluster_polygons(viirs_data, side_length=350, max_distance=750):
    """
    Merge squares into clusters and return a dictionary of merged polygons.
    Squares within max_distance meters of each other will be connected.
    """
    # Group points by cluster
    cluster_groups = viirs_data.groupby('fire_id')
    merged_clusters = {}

    for fire_id, cluster_data in cluster_groups:
        if len(cluster_data) == 1:
            # If only one point in cluster, just create a square
            square = create_square(
                cluster_data.iloc[0]['latitude'], 
                cluster_data.iloc[0]['longitude'], 
                side_length
            )
            merged_clusters[fire_id] = square
            continue

        # Get connections between nearby points
        connections = connect_nearby_squares(cluster_data, max_distance)

        # Create squares and connection polygons
        polygons = []

        # Create squares for each point
        for _, row in cluster_data.iterrows():
            square = create_square(row['latitude'], row['longitude'], side_length)
            polygons.append(square)

        # Create connection rectangles between nearby squares
        for i, j in connections:
            point1 = cluster_data.iloc[i]
            point2 = cluster_data.iloc[j]

            # Create a rectangle that connects the two squares
            line = LineString([
                (point1['longitude'], point1['latitude']),
                (point2['longitude'], point2['latitude'])
            ])
            # Buffer the line to create a rectangle
            # The buffer size is half the side length to match the squares
            connection = line.buffer(side_length/111320/2)
            polygons.append(connection)

        # Merge all polygons for this cluster
        merged_clusters[fire_id] = unary_union(polygons)

    return merged_clusters

def polygon_to_folium_coords(polygon):
    """Convert Shapely polygon coordinates to format needed by Folium"""
    if polygon.geom_type == 'MultiPolygon':
        return [[[coord[1], coord[0]] for coord in part.exterior.coords] 
                for part in polygon.geoms]
    else:
        return [[coord[1], coord[0]] for coord in polygon.exterior.coords]




In [6]:
# map centerd on Calabria, Italy
m = folium.Map(max_bounds=True)
bounds = [[37.988279,15.604679], [38.021622,15.90]]
m.fit_bounds(bounds=bounds)

# Add the Stadia Alidade Satellite tile layer
tile_provider = xyz.Stadia.AlidadeSatellite
api_key = os.getenv("STADIA_API_KEY")
tile_provider["url"] = tile_provider["url"] + "?api_key={api_key}"
#Update the URL to include the API key placeholder
tile_provider["url"] = tile_provider["url"] + "?api_key={api_key}"

#Create the folium TileLayer, specifying the API key
folium.TileLayer(
    tiles=tile_provider.build_url(api_key=api_key),
    attr=tile_provider.attribution,
    name=tile_provider.name,
    max_zoom=tile_provider.max_zoom,
    detect_retina=True
).add_to(m)

# Merge squares into cluster polygons
merged_clusters = merge_cluster_polygons(viirs, side_length=375)

# Add merged polygons to map
for fire_id, merged_polygon in merged_clusters.items():
    color = id_to_color[fire_id]
    coords = polygon_to_folium_coords(merged_polygon)
    start_date = viirs.loc[viirs['fire_id'] == fire_id, 'datetime'].iloc[0]
    end_date = viirs.loc[viirs['fire_id'] == fire_id, 'datetime'].iloc[-1]
    
    if isinstance(coords[0][0], list):  # MultiPolygon case
        for poly_coords in coords:
            folium.Polygon(
                locations=poly_coords,
                color=color,
                fill=True,
                fill_color=color,
                tooltip=f"{str(fire_id)} /\ {start_date.strftime('%m-%d')} - {end_date.strftime('%m-%d')}",
                fill_opacity=0.7
            ).add_to(m)
    else:  # Single polygon case
        folium.Polygon(
            locations=coords,
            color=color,
            fill=True,
            fill_color=color,
            tooltip=f"{str(fire_id)} /\ {start_date.strftime('%m-%d')} - {end_date.strftime('%m-%d')}",
            fill_opacity=0.7
        ).add_to(m)

m


In [9]:
# Set up folder for storing images
output_folder = './calabria_2012_summer_v2'
os.makedirs(output_folder, exist_ok=True)

# Convert datetime column to actual datetime objects if they aren't already
viirs['datetime'] = pd.to_datetime(viirs['datetime'], errors='coerce')

# Set up Selenium with Chrome in headless mode
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

# Collect screenshot paths for video creation
screenshot_paths = []

# Keep track of the latest version of each cluster
latest_clusters = {}

try:
    for index, row in viirs.iterrows():
        # Create a new map and fit bounds
        m = folium.Map(max_bounds=True)
        m.fit_bounds(bounds=bounds)
        #Create the folium TileLayer, specifying the API key
        folium.TileLayer(
            tiles=tile_provider.build_url(api_key=api_key),
            attr=tile_provider.attribution,
            name=tile_provider.name,
            max_zoom=tile_provider.max_zoom,
            detect_retina=True
        ).add_to(m)

        # Merge squares into cluster polygons
        current_viirs = viirs[viirs['datetime'] <= row['datetime']]
        merged_clusters = merge_cluster_polygons(current_viirs)

        # Add polygons to the map
        for cluster_id, merged_polygon in merged_clusters.items():
            # Only add the latest version of the cluster
            # if cluster_id not in latest_clusters or current_viirs.loc[current_viirs['cluster_id'] == cluster_id, 'datetime'].max() == row['datetime']:
            color = id_to_color[cluster_id]
            coords = polygon_to_folium_coords(merged_polygon)
            latest_clusters[cluster_id] = (color, coords)

            if isinstance(coords[0][0], list):  # MultiPolygon case
                for poly_coords in coords:
                    folium.Polygon(
                        locations=poly_coords,
                        color=color,
                        fill=True,
                        fill_color=color,
                        tooltip=str(cluster_id)
                    ).add_to(m)
            else:  # Single polygon case
                folium.Polygon(
                    locations=coords,
                    color=color,
                    fill=True,
                    fill_color=color,
                    tooltip=str(cluster_id)
                ).add_to(m)

        # Save map as an HTML file
        map_path = 'temp_map.html'
        m.save(map_path)

        # Open the map HTML file with Selenium
        driver.get("file://" + os.path.abspath(map_path))
        time.sleep(3)  # Wait for the map to load fully

        # Take a screenshot and save it
        screenshot = driver.get_screenshot_as_png()
        img = Image.open(io.BytesIO(screenshot))
        img_path = os.path.join(output_folder, f"{row['datetime'].strftime('%Y-%m-%d_%H-%M-%S')}.png")
        img.save(img_path)
        screenshot_paths.append(img_path)

finally:
    driver.quit()

KeyboardInterrupt: 

In [90]:
# Video creation
video_path = "./calabria_2012_summer_timelapse_v2.mp4"
frame_rate = 1.5  # 1 second per image

# Get dimensions from the first screenshot
if screenshot_paths:
    first_image = Image.open(screenshot_paths[0])
    width, height = first_image.size

    # Define video codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(video_path, fourcc, frame_rate, (width, height))

    # Add each screenshot to the video
    for img_path in screenshot_paths:
        frame = cv2.imread(img_path)
        video_writer.write(frame)

    video_writer.release()
else:
    print("No screenshots found. Video creation skipped.")